In [69]:
!pip install -U openai langchain ipywidgets unstructured chromadb  tiktoken

In [200]:
import os
from datetime import datetime

import openai
from IPython.display import HTML, display
from ipywidgets import widgets
from langchain.chains import ConversationalRetrievalChain

from langchain.document_loaders import DirectoryLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
   MessagesPlaceholder,
    SystemMessagePromptTemplate,
)
from langchain.chains import LLMChain
from langchain.schema import HumanMessage, SystemMessage


from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

## We can also use  ChatMessageHistory
## Use ConversationBufferWindowMemory to reduce context window
## Entity to exract context 



# Uses your OpenAI API Key
if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")

os.environ["OPENAI_API_KEY"] = openai_api_key

In [190]:
#loader = DirectoryLoader("documents/", glob="*.txt")
#txt_docs = loader.load_and_split()

llm = ChatOpenAI(model_name="gpt-4", temperature=0.2)
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            """You are a Product Manager Case Interviewer specializes in creating a realistic tech interview environment by posing a one line direct question such as "How would you improve Google Maps?" And reminder that users can use hints.  The chatbot encourages users to independently develop and articulate their ideas, maintaining the authenticity of an interview scenario. When a user requests a hint, the response is brief and minimally directive, offering just enough guidance. For clarifying questions about specifics like location, business objective, or audience population, the chatbot will provide a straightforward answer, selecting one possibility and presenting it without additional hints or guidance. This approach ensures a focused and efficient interview mock experience."""
        ),
        # The `variable_name` here is what must align with memory
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)
# Notice that we `return_messages=True` to fit into the MessagesPlaceholder
# Notice that `"chat_history"` aligns with the MessagesPlaceholder name.
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)




In [191]:
conversation_output=conversation({"question": "hi"})




> Entering new LLMChain chain...
Prompt after formatting:
System: You are a Product Manager Case Interviewer specializes in creating a realistic tech interview environment by posing a one line direct question such as "How would you improve Google Maps?" And reminder that users can use hints.  The chatbot encourages users to independently develop and articulate their ideas, maintaining the authenticity of an interview scenario. When a user requests a hint, the response is brief and minimally directive, offering just enough guidance. For clarifying questions about specifics like location, business objective, or audience population, the chatbot will provide a straightforward answer, selecting one possibility and presenting it without additional hints or guidance. This approach ensures a focused and efficient interview mock experience.
Human: hi

> Finished chain.


In [198]:
conversation({"question": "I have done this question before"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a Product Manager Case Interviewer specializes in creating a realistic tech interview environment by posing a one line direct question such as "How would you improve Google Maps?" And reminder that users can use hints.  The chatbot encourages users to independently develop and articulate their ideas, maintaining the authenticity of an interview scenario. When a user requests a hint, the response is brief and minimally directive, offering just enough guidance. For clarifying questions about specifics like location, business objective, or audience population, the chatbot will provide a straightforward answer, selecting one possibility and presenting it without additional hints or guidance. This approach ensures a focused and efficient interview mock experience.
Human: hi
AI: Hello! Let's get started. How would you improve Google Maps?
Human: I have done this question before
AI: No problem, let's try a different o

{'question': 'I have done this question before',
 'chat_history': [HumanMessage(content='hi'),
  AIMessage(content="Hello! Let's get started. How would you improve Google Maps?"),
  HumanMessage(content='I have done this question before'),
  AIMessage(content="No problem, let's try a different one. How would you improve the user experience of Amazon's Alexa?"),
  HumanMessage(content='Hello'),
  AIMessage(content="Hello! Let's dive into the question. How would you improve the user experience of Amazon's Alexa?"),
  HumanMessage(content='sounds good'),
  AIMessage(content="Great! Please share your thoughts on how you would improve the user experience of Amazon's Alexa."),
  HumanMessage(content='i have actually done this question before'),
  AIMessage(content="Understood. Let's try another one. How would you improve the functionality of Netflix's recommendation algorithm?"),
  HumanMessage(content='few clarification question:  can I assume we are improving in US, since this will simplif

In [192]:
conversation({"question": "I have done this question before"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a Product Manager Case Interviewer specializes in creating a realistic tech interview environment by posing a one line direct question such as "How would you improve Google Maps?" And reminder that users can use hints.  The chatbot encourages users to independently develop and articulate their ideas, maintaining the authenticity of an interview scenario. When a user requests a hint, the response is brief and minimally directive, offering just enough guidance. For clarifying questions about specifics like location, business objective, or audience population, the chatbot will provide a straightforward answer, selecting one possibility and presenting it without additional hints or guidance. This approach ensures a focused and efficient interview mock experience.
Human: hi
AI: Hello! Let's get started. How would you improve Google Maps?
Human: I have done this question before

> Finished chain.


{'question': 'I have done this question before',
 'chat_history': [HumanMessage(content='hi'),
  AIMessage(content="Hello! Let's get started. How would you improve Google Maps?"),
  HumanMessage(content='I have done this question before'),
  AIMessage(content="No problem, let's try a different one. How would you improve the user experience of Amazon's Alexa?")],
 'text': "No problem, let's try a different one. How would you improve the user experience of Amazon's Alexa?"}

In [193]:
conversation_output["text"]

"Hello! Let's get started. How would you improve Google Maps?"

In [194]:
%%html
<link rel="stylesheet" 
      href="https://cdn.jsdelivr.net/npm/bootstrap@4.5.3/dist/css/bootstrap.min.css" 
      integrity="sha384-TX8t27EcRE3e/ihU7zmQxVncDAy5uIKz4rEkgIXeMed4M0jlfIDPvg6uqKI2xXr2" 
      crossorigin="anonymous">
<style>
    body{margin-top:20px;}

    .chat-message-left,
    .chat-message-right {
        display: flex;
        flex-shrink: 0
    }

    .chat-message-left {
        margin-right: auto
    }

    .chat-message-right {
        flex-direction: row-reverse;
        margin-left: auto
    }
</style>

In [195]:

chat_history = []


def text_eventhandler(*args):
    # Needed bc when we "reset" the text input
    # it fires instantly another event since
    # we "changed" it's value to ""
    if args[0]["new"] == "":
        return

    # Show loading animation
    loading_bar.layout.display = "block"

    # Get question
    question = args[0]["new"]

    # Reset text field
    args[0]["owner"].value = ""

    # Formatting question for output
    q = (
        f'<div class="chat-message-right pb-4"><div>'
        + f'<img src="images/bear.png" class="rounded-circle mr-1" width="40" height="40">'
        + f'<div class="text-muted small text-nowrap mt-2">{datetime.now().strftime("%H:%M:%S")}</div></div>'
        + '<div class="flex-shrink-1 bg-light rounded py-2 px-3 ml-3">'
        + f'<div class="font-weight-bold mb-1">You</div>{question}</div>'
    )

    # Display formatted question
    output.append_display_data(HTML(q))

    try:
        answer = conversation({"question": f"{question}"})["text"]
        #answer = response["text"]
        chat_history.append((question, answer))
    except Exception as e:
        print("Question:", question)
        print("Request structure:", {"question": question})
        answer = "<b>Error:</b> " + str(e) +  " start " + question  + " end Charles" + type(question)

    # Formatting answer for output
    # Replacing all $ otherwise matjax would format them in a strange way
    answer_formatted = answer.replace("$", r"\$")
    a = (
        f'<div class="chat-message-left pb-4"><div>'
        + f'<img src="images/cat.png" class="rounded-circle mr-1" width="40" height="40">'
        + f'<div class="text-muted small text-nowrap mt-2">{datetime.now().strftime("%H:%M:%S")}</div></div>'
        + '<div class="flex-shrink-1 bg-light rounded py-2 px-3 ml-3">'
        + f'<div class="font-weight-bold mb-1">LLM</div>{answer_formatted}</div>'
    )

    # Turn off loading animation
    loading_bar.layout.display = "none"

    output.append_display_data(HTML(a))

In [196]:
in_text = widgets.Text()
in_text.continuous_update = False
in_text.observe(text_eventhandler, "value")
output = widgets.Output()

file = open("images/loading.gif", "rb")
image = file.read()
loading_bar = widgets.Image(
    value=image, format="gif", width="20", height="20", layout={"display": "None"}
)

In [197]:
display(
    widgets.HBox(
        [output],
        layout=widgets.Layout(
            width="100%",
            max_height="500px",
            display="inline-flex",
            flex_flow="column-reverse",
        ),
    )
)


display(
    widgets.Box(
        children=[loading_bar, in_text],
        layout=widgets.Layout(display="flex", flex_flow="row"),
    )
)

Box(children=(Image(value=b'GIF89a\xc8\x00\xc8\x00\xf7\x00\x00;Ch\x83\x90\xb7\xcf\xdc\xe8\xda\xec\xf1\xf1\xf2\…